# Table of Contents

[Import library](#import-libray)

[Start Spark](#spark-session)

[Staging customer dim](#stg_customer_dim)

# Import libray

In [1]:
import os

In [2]:
os.environ['HADOOP_USER_NAME'] = 'hadoopuser'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-spark_2.12:3.3.0,org.apache.hadoop:hadoop-aws:3.3.4 pyspark-shell'

In [3]:
import pyspark
import glob

from delta import *

from delta.tables import DeltaTable
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pprint import pprint

# Spark session

In [4]:
builder = (
    pyspark
    .sql.SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:29000") \
    .config("spark.hadoop.fs.s3a.access.key", "lLZ6r6vDyQvxq5WaTJ9w")
    .config("spark.hadoop.fs.s3a.secret.key", "WkxauVOq61tg0DnFQNSNnI8TbKfl9qxHBK3zILZ8")
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Logging
    # .config("spark.log.level", "DEBUG")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext # spark context

:: loading settings :: url = jar:file:/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ad/.ivy2/cache
The jars for the packages stored in: /home/ad/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2a8fbd6-2884-4ea4-93fa-514e22a73e77;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 90ms :: artifacts dl 3ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-

In [13]:
DeltaTable.forPath(
    spark, 
    "s3a://cdc-events/raw-supplier-cdc/"
).history().show(truncate=False)

+-------+-------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                |userMetadata|engineInfo                         |
+-------+-------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------+------------+-----------------------------------+
|200    |2025-05-21 11:12:48|NULL  |NULL    |WRITE    |{mode -> Append, partitionBy -> []}|NULL|NULL    |NULL     |199        |Serializable  |true         |{numFiles -> 1, numOutputRows -

# raw_supplier

In [5]:
raw_supplier_dt = DeltaTable.forPath(spark, 's3a://cdc-events/raw-supplier')

25/05/23 15:34:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
change_df = (
    spark.read
    .format("delta")
    .option("readChangeFeed", "true")
    .option("startingVersion", 0) # or startingTimestamp
    .load('s3a://cdc-events/raw-supplier')
)

change_df.show(truncate=False)


+---------+-------------------------+----------------------------------------+---------------+---------+-------------------------+-------------+-------------------+-------------+-------------------+-----------+--------+----------------+---------------+-----------------------+
|s_suppkey|s_name                   |s_address                               |s_phone        |s_acctbal|s_nationkey              |ts_ms        |create_ts          |delete_ts_ms |delete_ts          |lsn        |batch_id|_change_type    |_commit_version|_commit_timestamp      |
+---------+-------------------------+----------------------------------------+---------------+---------+-------------------------+-------------+-------------------+-------------+-------------------+-----------+--------+----------------+---------------+-----------------------+
|1000001  |New Supplier             |Ho Chi Minh city, Viet Nam              |27-918-335-1736|2049.4   |17                       |1747647591782|2025-05-19 16:39:51|NULL 

In [11]:
spark.sql(
    "select min(_commit_timestamp), max(_commit_timestamp) from table_changes_by_path('s3a://cdc-events/raw-supplier', 0)"
).collect()[0][0]

datetime.datetime(2025, 5, 21, 16, 6, 1, 201000)

# raw_supplier_cdc

In [14]:
raw_supplier_cdc_dt = DeltaTable.forPath(spark, 's3a://cdc-events/raw-supplier-cdc/')

raw_supplier_cdc_df = raw_supplier_cdc_dt.toDF()
raw_supplier_cdc_df.show()

+-----------------+--------------+-----------------+-------------------+---------------+-----------------+-----------------+----------------+--------------------+--------------------+------------------+---------------+----------------+--------------------+-------------+----------------+-------------------+------------+-----------+-------------+---+-------------+----------------+-------------------+--------+
|before__s_suppkey|before__s_name|before__s_address|before__s_nationkey|before__s_phone|before__s_acctbal|before__s_comment|after__s_suppkey|       after__s_name|    after__s_address|after__s_nationkey| after__s_phone|after__s_acctbal|    after__s_comment|source__ts_ms|   source__ts_us|      source__ts_ns|source__txId|source__lsn|source__table| op|        ts_ms|           ts_us|              ts_ns|batch_id|
+-----------------+--------------+-----------------+-------------------+---------------+-----------------+-----------------+----------------+--------------------+----------------

In [17]:
spark.sql("""
    select
        *
    from delta.`s3a://cdc-events/raw-supplier-cdc/`
    where op = 'd'
""").show()

+-----------------+--------------+-----------------+-------------------+---------------+-----------------+-----------------+----------------+-------------+----------------+------------------+--------------+----------------+----------------+-------------+----------------+-------------------+------------+-----------+-------------+---+-------------+----------------+-------------------+--------+
|before__s_suppkey|before__s_name|before__s_address|before__s_nationkey|before__s_phone|before__s_acctbal|before__s_comment|after__s_suppkey|after__s_name|after__s_address|after__s_nationkey|after__s_phone|after__s_acctbal|after__s_comment|source__ts_ms|   source__ts_us|      source__ts_ns|source__txId|source__lsn|source__table| op|        ts_ms|           ts_us|              ts_ns|batch_id|
+-----------------+--------------+-----------------+-------------------+---------------+-----------------+-----------------+----------------+-------------+----------------+------------------+--------------+----

In [12]:
spark.sql(f"select max(source__ts_ms) from delta.`s3a://cdc-events/raw-supplier-cdc/`").collect()[0][0]

1747647623499

In [ ]:
# get max date from the target table
# created_date_ts = (col(ts_ms) / 1000).cast(TimestampType())
start_date_ts = "select max(created_date_ts) from target"

# incremental query
sql = f"select * from delta.`s3a://cdc-events/raw-supplier-cdc/` where source__ts_ms >= {start_date_ts} limit 1000"

# raw_lineitem

In [ ]:
# raw_line_item_dt = DeltaTable.forPath(
#     spark,
#     "hdfs://localhost:9000/user/hadoopuser/raw/raw_lineitem"
# )

# raw_line_item_dt.optimize().executeZOrderBy(
#     "l_orderkey",
#     "l_partkey",
#     "l_suppkey"
# )

# stg_customer_dim

In [ ]:
# define table
table_path = "hdfs://localhost:9000/user/hadoopuser/stage/stg_customer_dim"
dt = DeltaTable.forPath(spark, table_path)

df = dt.toDF()
df.show()

In [ ]:
dt.history().show(truncate=False)

In [ ]:
dt.detail().show(truncate=False)

In [ ]:
df.count()

In [ ]:
dt.optimize().executeCompaction()

# stg_order_dim

In [ ]:
stg_order_dt = DeltaTable.forPath(
    spark, 
    "hdfs://localhost:9000/user/hadoopuser/stage/stg_order_dim"
)

In [ ]:
stg_order_dt.history().show(truncate=False)

In [ ]:
stg_order_dt.toDF().show(10)

## update schema
```sql
ALTER TABLE table_name 
ADD COLUMNS (col_name data_type [COMMENT col_comment] [FIRST|AFTER colA_name], ...)
```

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_order_dim`
    ADD COLUMNS (record_track_hash INTEGER)
""")

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_order_dim` 
    SET TBLPROPERTIES (
        'delta.minReaderVersion' = '2',
        'delta.minWriterVersion' = '5',
        'delta.columnMapping.mode' = 'name'
    )
""")

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_order_dim`
    DROP COLUMN o_comment
""")

# stg_part_dim

In [ ]:
stg_part_dt = DeltaTable.forPath(spark, 'hdfs://localhost:9000/user/hadoopuser/stage/stg_part_dim')

In [ ]:
stg_part_dt.history().show(truncate=False)

In [ ]:
stg_part_dt.toDF().show(truncate=False)

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_part_dim` 
    SET TBLPROPERTIES (
        'delta.minReaderVersion' = '2',
        'delta.minWriterVersion' = '5',
        'delta.columnMapping.mode' = 'name'
    )
""")

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_part_dim`
    ADD COLUMNS (record_track_hash INTEGER)
""")

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_part_dim`
    DROP COLUMN p_comment
""")

# stg_supplier_dim

In [ ]:
stg_supplier_dt = DeltaTable.forPath(
    spark,
    'hdfs://localhost:9000/user/hadoopuser/stage/stg_supplier_dim'
)

In [ ]:
stg_supplier_dt.history().show(truncate=False)

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_supplier_dim` 
    SET TBLPROPERTIES (
        'delta.minReaderVersion' = '2',
        'delta.minWriterVersion' = '5',
        'delta.columnMapping.mode' = 'name'
    )
""")

spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_supplier_dim`
    ADD COLUMNS (record_track_hash INTEGER)
""")

spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_supplier_dim`
    DROP COLUMN s_comment
""")


In [ ]:
stg_supplier_dt.toDF().show()

In [ ]:
stg_supplier_dt.toDF().where("record_active_flag = true").show()

# stg_line_item_fact

In [5]:
stg_line_item_dt = DeltaTable.forPath(
    spark,
    'hdfs://localhost:9000/user/hadoopuser/stage/stg_line_item_fact'
)

In [6]:
stg_line_item_dt.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_line_item_fact`
    ADD COLUMNS (
        __session_id CHAR(36)
    )
""")

DataFrame[]

In [ ]:
spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_line_item_fact` 
    SET TBLPROPERTIES (
        'delta.minReaderVersion' = '2',
        'delta.minWriterVersion' = '5',
        'delta.columnMapping.mode' = 'name'
    )
""")

spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_line_item_fact`
    ADD COLUMNS (
        sk_orderkey INTEGER AFTER sk_lineitemkey,
        sk_partkey INTEGER AFTER sk_lineitemkey,
        sk_suppkey INTEGER AFTER sk_lineitemkey,
        sk_custkey INTEGER AFTER sk_lineitemkey,
        record_track_hash INTEGER
    )
""")

spark.sql("""
    ALTER TABLE delta.`hdfs://localhost:9000/user/hadoopuser/stage/stg_line_item_fact`
    DROP COLUMNS (l_partkey, l_suppkey, o_custkey, l_comment)
""")


In [8]:
stg_line_item_dt.toDF().show(10, truncate=False)

+--------------+-----------+----------+----------+----------+----------+------------+-----------+---------------+--------------+-------------+------------+---------------+-------------+-----------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-------------------------+----------+------------------+-----------------+---------------+-----------------+------------------------------------+
|sk_lineitemkey|sk_orderkey|sk_partkey|sk_suppkey|sk_custkey|l_orderkey|l_linenumber|o_orderdate|o_orderpriority|o_shippriority|o_orderstatus|o_totalprice|o_clerk        |ps_supplycost|ps_availqty|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|l_shipinstruct           |l_shipmode|record_active_flag|record_start_date|record_end_date|record_track_hash|__session_id                        |
+--------------+-----------+----------+----------+----------+----------+------------+-----------+-

# Check key distribution in table

## dim_order

In [5]:
dim_order_df = (
    spark.read
    .format("delta")
    .load("s3a://data-warehouse/dim_order")
)

dim_order_df.show()

25/04/29 15:29:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/04/29 15:29:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+-------------+------------+-----------+---------------+---------------+--------------+------------------+-----------------+---------------+-----------------+
|sk_orderkey|o_orderkey|o_orderstatus|o_totalprice|o_orderdate|o_orderpriority|        o_clerk|o_shippriority|record_active_flag|record_start_date|record_end_date|record_track_hash|
+-----------+----------+-------------+------------+-----------+---------------+---------------+--------------+------------------+-----------------+---------------+-----------------+
|         28| 271192098|            O|    79590.51| 1997-02-12|1-URGENT       |Clerk#000049928|             0|              true|       2025-04-26|           NULL|       -587546875|
|         77| 271192099|            O|   107015.22| 1998-05-13|2-HIGH         |Clerk#000028896|             0|              true|       2025-04-26|           NULL|       -746102867|
|        126| 271192100|            O|    37755.82| 1995-08-06|3-MEDIUM       |Clerk#00005

# Analyze table

In [5]:
spark.sql(
    "ANALYZE TABLE delta.`hdfs://localhost:9000/user/hadoopuser/raw/raw_lineitem` COMPUTE STATISTICS"
)

AnalysisException: [NOT_SUPPORTED_COMMAND_FOR_V2_TABLE] ANALYZE TABLE is not supported for v2 tables.

# Stop Spark

In [ ]:
spark.stop()
# sc._gateway.jvm.System.exit(0)